In [69]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import os
from dotenv import load_dotenv
import unicodedata
from src.etl import load as lo #con jupyter
from src.eda import explore as exp  #con jupyter
from src.etl import extract as ex #con jupyter

In [70]:
load_dotenv()

True

In [71]:
#Llamamos a la variable de entorno
DB_NAME = os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST= os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [72]:
conn, cur = lo.crear_conexión()
#Comprobamos que la conexión está bien realizada:
cur.execute("SELECT version();")
cur.fetchone() #para que muestre el resultado de la query anterior
lo.cerrar_conexion(conn, cur)

# 1. Extracción de los datos de la BBDD

## Paises

In [73]:
query_extraccion = '''SELECT *
                                    FROM pais_destino'''        

In [74]:
lista_columnas = ['id_pais_destino','nombre_pais_destino', 'nombre_continente']
df_paises = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_paises

,id_pais_destino,nombre_pais_destino,nombre_continente
0,1,Albania,Europa
1,2,Alemania,Europa
2,3,Andorra,Europa
3,4,Angola,África
4,5,Antigua,América
...,...,...,...
183,184,Vietnam,Asia
184,185,Yemen,Asia
185,186,Zambia,África
186,187,Zanzibar,África


## Viajes localizados según página web

In [75]:
query_extraccion = '''SELECT pd.*,
                                                        v.*
                                                    FROM viaje v
                                                    LEFT JOIN combinacion_destino_viaje cdv ON v.id_viaje = cdv.id_viaje
                                                    LEFT JOIN pais_destino pd ON cdv.id_pais_destino =pd.id_pais_destino'''     

In [76]:
lista_columnas = ['id_pais_destino', 'nombre_pais_destino', 'nombre_continente', 'id_viaje', 'url_viaje', 
                  'nombre_viaje','duracion_dias', 'duracion_noches', 'id_itinerario', 'viaje_activo']

In [77]:
df_viajes_localizacion_pag_web = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viajes_localizacion_pag_web

,id_pais_destino,nombre_pais_destino,nombre_continente,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo
0,1,Albania,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027.0,True
1,2,Alemania,Europa,2,https://viajeonline.es.tui.com/es/idea/1463025...,Berlín,4,3,146.0,True
2,2,Alemania,Europa,3,https://viajeonline.es.tui.com/es/idea/7897512...,"Fly & Drive Austria, Entre Lagos y Montañas",8,7,872.0,True
3,2,Alemania,Europa,4,https://viajeonline.es.tui.com/es/idea/8404511...,Fly & Drive Carlomagno a su aire,10,9,166.0,True
4,2,Alemania,Europa,5,https://viajeonline.es.tui.com/es/idea/7906930...,Fly & Drive Tirol al Completo,8,7,1145.0,True
...,...,...,...,...,...,...,...,...,...,...
1765,97,Laponia,Europa,1449,https://viajeonline.es.tui.com/es/idea/2607847...,Laponia - Puente de Diciembre en Levi - desde ...,5,4,570.0,True
1766,97,Laponia,Europa,1450,https://viajeonline.es.tui.com/es/idea/2607811...,Laponia - Puente de Diciembre en Levi - desde ...,5,4,570.0,True
1767,115,Mexico,América,1451,https://viajeonline.es.tui.com/es/idea/2700326...,Ciudad de México y Holbox,9,7,273.0,True
1768,149,Riviera Maya,América,1451,https://viajeonline.es.tui.com/es/idea/2700326...,Ciudad de México y Holbox,9,7,273.0,True


In [78]:
df_viajes_localizacion_pag_web_agrupado_pais = df_viajes_localizacion_pag_web.groupby('id_pais_destino')[['id_viaje']].count().reset_index()
df_viajes_localizacion_pag_web_agrupado_pais

,id_pais_destino,id_viaje
0,1,5
1,2,20
2,5,2
3,6,1
4,8,26
...,...,...
121,181,1
122,182,4
123,184,43
124,187,23


In [79]:
len(df_viajes_localizacion_pag_web.id_pais_destino.unique())

126

## Viajes según localización ciudades

In [80]:
query_extraccion = '''SELECT pd.*,
	v.*,
	c.id_ciudad,
	c.nombre_ciudad
FROM viaje v
INNER JOIN itinerario i ON v.id_itinerario = i.id_itinerario
INNER JOIN ciudad_itinerario ci ON i.id_itinerario = ci.id_itinerario
INNER JOIN ciudad c ON ci.id_ciudad = c.id_ciudad
INNER JOIN pais_destino pd ON c.id_pais_destino =pd.id_pais_destino'''


In [81]:
lista_columnas = ['id_pais_destino', 'nombre_pais_destino', 'nombre_continente', 'id_viaje', 'url_viaje', 
                  'nombre_viaje','duracion_dias', 'duracion_noches', 'id_itinerario', 'viaje_activo', 'id_ciudad', 'nombre_ciudad']

In [82]:
df_viajes_localizacion_ciudad = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viajes_localizacion_ciudad

,id_pais_destino,nombre_pais_destino,nombre_continente,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo,id_ciudad,nombre_ciudad
0,119,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027,True,9,Bar
1,119,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027,True,8,Cetinje
2,119,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027,True,7,Budva
3,119,Montenegro,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027,True,6,Petrovac
4,1,Albania,Europa,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027,True,5,Shkoder
...,...,...,...,...,...,...,...,...,...,...,...,...
9110,77,India,Asia,1452,https://es.tui.com/viaje/india-y-maldivas-a-tu...,India y maldivas a tu alcance,11,9,343,True,264,Agra
9111,77,India,Asia,1452,https://es.tui.com/viaje/india-y-maldivas-a-tu...,India y maldivas a tu alcance,11,9,343,True,263,Fatehpur Sikri
9112,77,India,Asia,1452,https://es.tui.com/viaje/india-y-maldivas-a-tu...,India y maldivas a tu alcance,11,9,343,True,261,Jaipur
9113,77,India,Asia,1452,https://es.tui.com/viaje/india-y-maldivas-a-tu...,India y maldivas a tu alcance,11,9,343,True,900,Shahpura


In [83]:
len(df_viajes_localizacion_ciudad.nombre_pais_destino.unique())

116

In [84]:
df_viajes_localizacion_ciudad_agrupado_pais = df_viajes_localizacion_ciudad.groupby('id_pais_destino')[['id_viaje']].count().reset_index()
df_viajes_localizacion_ciudad_agrupado_pais

,id_pais_destino,id_viaje
0,1,19
1,2,84
2,6,7
3,8,116
4,9,19
...,...,...
111,180,49
112,181,3
113,182,18
114,184,349


## Ciudad

In [85]:
query_extraccion = '''SELECT *
FROM ciudad c'''

lista_columnas = ['id_ciudad', 'nombre_ciudad','id_pais_destino']
df_ciudad = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_ciudad

,id_ciudad,nombre_ciudad,id_pais_destino
0,1,Tirana,1
1,2,Durres,1
2,3,Berat,1
3,4,Kruja,1
4,5,Shkoder,1
...,...,...,...
2070,2071,Dilijan,9
2071,2072,Sanahin Tbilisi,65
2072,2073,Baku,65
2073,2074,Gobustan,56


In [86]:
df_ciudad_sin_pais = df_ciudad[pd.isnull(df_ciudad.id_pais_destino)].copy()
df_ciudad_sin_pais['nombre_pais_destino']=None

In [87]:
# Hay 10 ciudades que no tienen pais asociado en la BBDD - Corregir en próximos pasos

In [88]:
df_opencague = pd.read_pickle('../data/data_raw/geolocalizacion_api_total_ciudades.pkl')

In [89]:
df_opencague=df_opencague.rename(columns={'ciudad':'nombre_ciudad'})
df_opencague

,nombre_ciudad,continente_api,pais_api,latitud,longitud
0,Tirana,Europe,Albania,41.328148,19.818444
1,Dürres,Europe,Albania,41.313255,19.446235
2,Berat,Europe,Albania,40.701864,19.958076
3,Kruja,Europe,Albania,41.511251,19.792852
4,Shkodër,Europe,Albania,42.068137,19.512144
...,...,...,...,...,...
1471,Mesa Verde,North America,Estados Unidos de América,33.605858,-114.731072
1472,Great Sand,Oceania,Australia,-38.262747,144.775661
1473,Dunes,Europe,Francia,44.087932,0.769998
1474,Pueblo,North America,Estados Unidos de América,38.263995,-104.614187


In [90]:
ex.incorporar_información_df_original(df_ciudad_sin_pais,df_opencague,'nombre_ciudad','pais_api', 'nombre_pais_destino')

In [91]:
df_ciudad_sin_pais

,id_ciudad,nombre_ciudad,id_pais_destino,nombre_pais_destino


In [92]:
query_extraccion = "SELECT nombre_pais_destino, id_pais_destino FROM pais_destino"
df_pais_destino=exp.extraer_datos_bbdd_y_convertir_en_df(['nombre_pais_destino','id_pais_destino'],query_extraccion)

In [93]:
df_pais_destino

,nombre_pais_destino,id_pais_destino
0,Albania,1
1,Alemania,2
2,Andorra,3
3,Angola,4
4,Antigua,5
...,...,...
183,Vietnam,184
184,Yemen,185
185,Zambia,186
186,Zanzibar,187


In [94]:
ex.incorporar_información_df_original(df_ciudad_sin_pais,df_pais_destino,'nombre_pais_destino','id_pais_destino', 'id_pais_destino')

In [95]:
df_ciudad_sin_pais

,id_ciudad,nombre_ciudad,id_pais_destino,nombre_pais_destino


## Itinerario_ciudad

In [96]:
query_extraccion = '''SELECT *
FROM ciudad_itinerario'''

lista_columnas = ['id_ciudad', 'id_itinerario']
df_ciudad_itinerario = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_ciudad_itinerario

,id_ciudad,id_itinerario
0,1,1027
1,2,1027
2,3,1027
3,4,1027
4,5,1027
...,...,...
7530,900,343
7531,261,343
7532,263,343
7533,264,343


In [97]:
df_ciudad_itinerario[df_ciudad_itinerario.id_ciudad == 739]

,id_ciudad,id_itinerario
1669,739,857
7517,739,570


## Viajes

In [98]:
query_extraccion = '''SELECT *
FROM viaje v'''

lista_columnas = ['id_viaje', 'url_viaje', 'nombre_viaje', 'duracion_dias', 
                  'duracion_noches','id_itinerario', 'viaje_activo']
df_viajes = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viajes

,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo
0,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027.0,True
1,2,https://viajeonline.es.tui.com/es/idea/1463025...,Berlín,4,3,146.0,True
2,3,https://viajeonline.es.tui.com/es/idea/7897512...,"Fly & Drive Austria, Entre Lagos y Montañas",8,7,872.0,True
3,4,https://viajeonline.es.tui.com/es/idea/8404511...,Fly & Drive Carlomagno a su aire,10,9,166.0,True
4,5,https://viajeonline.es.tui.com/es/idea/7906930...,Fly & Drive Tirol al Completo,8,7,1145.0,True
...,...,...,...,...,...,...,...
1447,1448,https://viajeonline.es.tui.com/es/idea/2607664...,Laponia - Puente de Diciembre en Levi - desde ...,5,4,570.0,True
1448,1449,https://viajeonline.es.tui.com/es/idea/2607847...,Laponia - Puente de Diciembre en Levi - desde ...,5,4,570.0,True
1449,1450,https://viajeonline.es.tui.com/es/idea/2607811...,Laponia - Puente de Diciembre en Levi - desde ...,5,4,570.0,True
1450,1451,https://viajeonline.es.tui.com/es/idea/2700326...,Ciudad de México y Holbox,9,7,273.0,True


## Precio de los viajes

In [99]:
query_extraccion = '''SELECT v.*,
	precio_viaje,
	fecha_precio_viaje
FROM viaje v
INNER JOIN precio_viaje pv ON v.id_viaje = pv.id_viaje'''

In [100]:
lista_columnas = [ 'id_viaje', 'url_viaje', 'nombre_viaje','duracion_dias', 'duracion_noches', 'id_itinerario', 
                  'viaje_activo', 'precio_viaje', 'fecha_precio_viaje']

In [101]:
df_viaje_precios = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viaje_precios

,id_viaje,url_viaje,nombre_viaje,duracion_dias,duracion_noches,id_itinerario,viaje_activo,precio_viaje,fecha_precio_viaje
0,1,https://es.tui.com/viaje/albania-y-montenegro,Albania y Montenegro,8,7,1027.0,True,1755,2025-04-07
1,2,https://viajeonline.es.tui.com/es/idea/1463025...,Berlín,4,3,146.0,True,708,2025-04-07
2,3,https://viajeonline.es.tui.com/es/idea/7897512...,"Fly & Drive Austria, Entre Lagos y Montañas",8,7,872.0,True,827,2025-04-07
3,4,https://viajeonline.es.tui.com/es/idea/8404511...,Fly & Drive Carlomagno a su aire,10,9,166.0,True,857,2025-04-07
4,5,https://viajeonline.es.tui.com/es/idea/7906930...,Fly & Drive Tirol al Completo,8,7,1145.0,True,894,2025-04-07
...,...,...,...,...,...,...,...,...,...
48626,1381,https://es.tui.com/viaje/paisajes-y-tribus-de-...,Paisajes y Tribus de Vietnam y Camboya,17,14,458.0,True,3284,2025-05-20
48627,1382,https://es.tui.com/viaje/paisajes-y-tribus-de-...,Paisajes y Tribus de Vietnam y Phu Quoc,17,14,452.0,True,3310,2025-05-20
48628,1383,https://es.tui.com/viaje/paisajes-y-tribus-de-...,Paisajes y Tribus de Vietnam y Nha Trang,17,14,451.0,True,3450,2025-05-20
48629,1384,https://es.tui.com/viaje/zimbabue-salvaje,Zimbabue Salvaje,11,8,234.0,True,6539,2025-05-20


## Viajes con cambios de precios

In [102]:
query_extraccion = '''SELECT v.id_viaje,
	count(DISTINCT precio_viaje) as num_precios_ofertados
FROM viaje v
INNER JOIN precio_viaje pv ON v.id_viaje = pv.id_viaje
GROUP BY v.id_viaje
HAVING count(DISTINCT precio_viaje)>1;'''

In [103]:
lista_columnas = [ 'id_viaje', 'num_precios_ofertados']

In [104]:
df_viaje_con_precios_distintos = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_viaje_con_precios_distintos

,id_viaje,num_precios_ofertados
0,11,2
1,12,2
2,15,2
3,16,2
4,17,2
5,19,2
6,20,2
7,21,2
8,23,2
9,24,2


## Turismo emisor

In [105]:
query_extraccion = '''SELECT pd.*,
	ccaa_origen,
	num_turistas,
	num_pernoctaciones,
	ano,
	mes
FROM turismo_emisor te 
INNER JOIN pais_destino pd ON te.id_pais_destino = pd.id_pais_destino
ORDER BY ano,mes ;'''

In [106]:
lista_columnas = [ 'id_pais_destino', 'nombre_pais_destino', 'nombre_continente', 'ccaa_origen','num_turistas', 'num_pernoctaciones', 'ano', 'mes']

In [107]:
df_turismo_emisor = exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_turismo_emisor

,id_pais_destino,nombre_pais_destino,nombre_continente,ccaa_origen,num_turistas,num_pernoctaciones,ano,mes
0,8,Argentina,América,Andalucía,419,3616,2019,7
1,22,Bolivia,América,Andalucía,61,486,2019,7
2,25,Brasil,América,Andalucía,277,2214,2019,7
3,32,Canada,América,Andalucía,483,6159,2019,7
4,36,Chile,América,Andalucía,242,2068,2019,7
...,...,...,...,...,...,...,...,...
71092,159,Seychelles,África,Total Nacional,146,1105,2024,12
71093,165,Sudafrica,África,Total Nacional,986,9019,2024,12
71094,171,Tanzania,África,Total Nacional,590,5237,2024,12
71095,175,Tunez,África,Total Nacional,975,5012,2024,12


In [108]:
#conn.rollback()

In [109]:
df_turismo_emisor_agrupado = df_turismo_emisor.groupby('nombre_pais_destino')[['num_turistas', 'num_pernoctaciones']].sum()
df_turismo_emisor_agrupado

,num_turistas,num_pernoctaciones
nombre_pais_destino,,
Albania,180212,1115910
Alemania,11949860,68516775
Andorra,9323631,24769193
Angola,9669,79896
Antigua,220,1900
...,...,...
Venezuela,106829,803330
Vietnam,130169,1477676
Yemen,222,2128



## Consulta por paises con los viajes demandados por el turismo Español y el num. viajes ofertados por TUI con la localización de la pag. web para ellos.

In [110]:
query_extraccion = '''WITH cte_turismo_emisor_resumen AS (
  SELECT 
    id_pais_destino,
    SUM(num_turistas) AS suma_turistas,
    SUM(num_pernoctaciones) AS suma_pernoctaciones
  FROM turismo_emisor
  GROUP BY id_pais_destino
)

SELECT pd.nombre_continente AS continente,
	pd.nombre_pais_destino AS pais_destino, 
  te.suma_turistas,
  te.suma_pernoctaciones,
   (te.suma_pernoctaciones / te.suma_turistas) AS promedio_pernoctaciones_turismo_emisor,
	count(DISTINCT v.id_viaje) AS num_viajes_ofertados_tui,
	round(avg(v.duracion_noches),0) AS promedio_pernoctaciones_viajes_tui	
FROM cte_turismo_emisor_resumen te 
LEFT JOIN pais_destino pd ON te.id_pais_destino =pd.id_pais_destino
LEFT JOIN combinacion_destino_viaje cdv ON pd.id_pais_destino =cdv.id_pais_destino
LEFT JOIN viaje v ON cdv.id_viaje = v.id_viaje 
GROUP BY 1,2,3,4,5
ORDER BY pais_destino;'''
lista_columnas = [ 'nombre_continente', 'nombre_pais_destino', 'suma_turistas', 'suma_pernoctaciones','promedio_pernoctaciones_turismo_emisor', 'num_viajes_ofertados_tui', 'promedio_pernoctaciones_viajes_tui']
df_combi_tur_emisor_tui_web= exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_combi_tur_emisor_tui_web

,nombre_continente,nombre_pais_destino,suma_turistas,suma_pernoctaciones,promedio_pernoctaciones_turismo_emisor,num_viajes_ofertados_tui,promedio_pernoctaciones_viajes_tui
0,Europa,Albania,180212,1115910,6,5,9
1,Europa,Alemania,11949860,68516775,5,20,7
2,Europa,Andorra,9323631,24769193,2,0,None
3,África,Angola,9669,79896,8,0,None
4,América,Antigua,220,1900,8,2,7
...,...,...,...,...,...,...,...
153,América,Venezuela,106829,803330,7,0,None
154,Asia,Vietnam,130169,1477676,11,43,11
155,Asia,Yemen,222,2128,9,0,None
156,África,Zambia,382,2881,7,0,None


## Consulta por paises con los viajes demandados por el turismo Español y el num. viajes ofertados por TUI pero teniendo en cuenta las ciudades:

In [111]:
query_extraccion = '''WITH cte_turismo_emisor_resumen AS (
  SELECT 
    id_pais_destino,
    SUM(num_turistas) AS suma_turistas,
    SUM(num_pernoctaciones) AS suma_pernoctaciones
  FROM turismo_emisor
  GROUP BY id_pais_destino
)


SELECT pd.nombre_continente AS continente,
	pd.nombre_pais_destino AS pais_destino, 
  te.suma_turistas,
  te.suma_pernoctaciones,
   (te.suma_pernoctaciones / te.suma_turistas) AS promedio_pernoctaciones_turismo_emisor,
	count(DISTINCT v.id_viaje) AS num_viajes_ofertados_tui,
	round(avg(v.duracion_noches),0) AS promedio_pernoctaciones_viajes_tui	
FROM cte_turismo_emisor_resumen te 
LEFT JOIN pais_destino pd ON te.id_pais_destino =pd.id_pais_destino
LEFT JOIN ciudad c  ON pd.id_pais_destino =c.id_pais_destino
LEFT JOIN ciudad_itinerario ci ON c.id_ciudad =ci.id_ciudad
LEFT JOIN itinerario i ON ci.id_itinerario = i.id_itinerario
LEFT JOIN viaje v ON i.id_itinerario  = v.id_itinerario 
GROUP BY 1,2,3,4,5
ORDER BY pais_destino;'''
lista_columnas = [ 'nombre_continente', 'nombre_pais_destino', 'suma_turistas', 'suma_pernoctaciones','promedio_pernoctaciones_turismo_emisor', 'num_viajes_ofertados_tui', 'promedio_pernoctaciones_viajes_tui']
df_combi_tur_emisor_tui_ciudad= exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_combi_tur_emisor_tui_ciudad

,nombre_continente,nombre_pais_destino,suma_turistas,suma_pernoctaciones,promedio_pernoctaciones_turismo_emisor,num_viajes_ofertados_tui,promedio_pernoctaciones_viajes_tui
0,Europa,Albania,180212,1115910,6,5,8
1,Europa,Alemania,11949860,68516775,5,19,7
2,Europa,Andorra,9323631,24769193,2,0,None
3,África,Angola,9669,79896,8,0,None
4,América,Antigua,220,1900,8,0,None
...,...,...,...,...,...,...,...
153,América,Venezuela,106829,803330,7,0,None
154,Asia,Vietnam,130169,1477676,11,43,12
155,Asia,Yemen,222,2128,9,0,None
156,África,Zambia,382,2881,7,0,None


## Consulta por paises con los viajes ofertados por tui y los demandados por el turismo Español teniendo en cuenta las ciudades:

In [112]:
query_extraccion = '''
WITH cte_turismo_emisor_resumen AS (
  SELECT 
    id_pais_destino,
    SUM(num_turistas) AS suma_turistas,
    SUM(num_pernoctaciones) AS suma_pernoctaciones
  FROM turismo_emisor
  GROUP BY id_pais_destino
)

SELECT pd.nombre_continente AS continente,
	pd.nombre_pais_destino AS pais_destino, 
	count(DISTINCT v.id_viaje) AS num_viajes_ofertados_tui,
	round(avg(v.duracion_noches),0) AS promedio_pernoctaciones_viajes_tui,
    te.suma_turistas,
  te.suma_pernoctaciones,
  (te.suma_pernoctaciones / te.suma_turistas) AS promedio_pernoctaciones_turismo_emisor
FROM viaje v 
LEFT JOIN itinerario i ON v.id_itinerario = i.id_itinerario
LEFT JOIN ciudad_itinerario ci  ON i.id_itinerario = ci.id_ciudad 
LEFT JOIN ciudad c ON ci.id_ciudad = c.id_ciudad
LEFT JOIN pais_destino pd ON c.id_pais_destino = pd.id_pais_destino
LEFT JOIN cte_turismo_emisor_resumen te ON pd.id_pais_destino = te.id_pais_destino
GROUP BY 1,2,5,6,7
ORDER BY pais_destino ASC;'''
lista_columnas = [ 'nombre_continente', 'nombre_pais_destino', 'num_viajes_ofertados_tui', 'promedio_pernoctaciones_viajes_tui','suma_turistas', 'suma_pernoctaciones', 'promedio_pernoctaciones_turismo_emisor']
df_combi_tui_ciudad_tur_emisor= exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_combi_tui_ciudad_tur_emisor

,nombre_continente,nombre_pais_destino,num_viajes_ofertados_tui,promedio_pernoctaciones_viajes_tui,suma_turistas,suma_pernoctaciones,promedio_pernoctaciones_turismo_emisor
0,Europa,Albania,5,9,180212.0,1115910.0,6.0
1,Europa,Alemania,24,8,11949860.0,68516775.0,5.0
2,América,Argentina,10,14,615534.0,6812392.0,11.0
3,América,Aruba,6,9,NaN,NaN,NaN
4,Oceanía,Australia,63,10,89860.0,1047125.0,11.0
...,...,...,...,...,...,...,...
89,América,Turks And Caicos,1,7,NaN,NaN,NaN
90,Asia,Turquia,3,9,2122162.0,11685594.0,5.0
91,Asia,Vietnam,6,9,130169.0,1477676.0,11.0
92,África,Zimbabue,7,10,3364.0,34893.0,10.0


## Consulta por paises con los viajes ofertados por tui y los demandados por el turismo Español según están registrados en la página de TUI:

In [113]:
query_extraccion = '''
WITH cte_turismo_emisor_resumen AS (
  SELECT 
    id_pais_destino,
    SUM(num_turistas) AS suma_turistas,
    SUM(num_pernoctaciones) AS suma_pernoctaciones
  FROM turismo_emisor
  GROUP BY id_pais_destino
)

-- EJECUTO LA CONSULTA
SELECT 
  pd.nombre_continente AS continente,
  pd.nombre_pais_destino AS pais_destino, 
  COUNT(DISTINCT v.id_viaje) AS num_viajes_ofertados_tui,
  ROUND(AVG(v.duracion_noches), 0) AS promedio_pernoctaciones_viajes_tui,
  te.suma_turistas,
  te.suma_pernoctaciones,
   (te.suma_pernoctaciones / te.suma_turistas) AS promedio_pernoctaciones_turismo_emisor
FROM viaje v
LEFT JOIN combinacion_destino_viaje cdv ON v.id_viaje = cdv.id_viaje
LEFT JOIN pais_destino pd ON cdv.id_pais_destino = pd.id_pais_destino
LEFT JOIN cte_turismo_emisor_resumen te ON pd.id_pais_destino = te.id_pais_destino
GROUP BY 1, 2, 5, 6, 7
ORDER BY pais_destino;'''
lista_columnas = [ 'nombre_continente', 'nombre_pais_destino', 'num_viajes_ofertados_tui', 'promedio_pernoctaciones_viajes_tui','suma_turistas', 'suma_pernoctaciones', 'promedio_pernoctaciones_turismo_emisor']
df_combi_tui_web_tur_emisor_ciudad= exp.extraer_datos_bbdd_y_convertir_en_df(lista_columnas,query_extraccion)
df_combi_tui_web_tur_emisor_ciudad

,nombre_continente,nombre_pais_destino,num_viajes_ofertados_tui,promedio_pernoctaciones_viajes_tui,suma_turistas,suma_pernoctaciones,promedio_pernoctaciones_turismo_emisor
0,Europa,Albania,5,9,180212.0,1115910.0,6.0
1,Europa,Alemania,20,7,11949860.0,68516775.0,5.0
2,América,Antigua,2,7,220.0,1900.0,8.0
3,Asia,Arabia Saudi,1,7,150581.0,970902.0,6.0
4,América,Argentina,26,11,615534.0,6812392.0,11.0
...,...,...,...,...,...,...,...
121,América,Uruguay,1,7,84503.0,897617.0,10.0
122,Asia,Uzbekistan,4,7,41950.0,314907.0,7.0
123,Asia,Vietnam,43,11,130169.0,1477676.0,11.0
124,África,Zanzibar,23,8,NaN,NaN,NaN


# Creo DF con los paises el num. de viajes por web y el num viajes por ciudad (agrupando por pais).

In [114]:
df_paises_con_viajes_y_turismo = df_paises.copy()

In [115]:
df_paises_con_viajes_y_turismo

,id_pais_destino,nombre_pais_destino,nombre_continente
0,1,Albania,Europa
1,2,Alemania,Europa
2,3,Andorra,Europa
3,4,Angola,África
4,5,Antigua,América
...,...,...,...
183,184,Vietnam,Asia
184,185,Yemen,Asia
185,186,Zambia,África
186,187,Zanzibar,África


In [116]:
df_paises_con_viajes_y_turismo[['num_viajes_tui_web', 'num_viajes_segun_viaje_itinerario', 
                                'total_turistas','total_pernoctaciones']] = None

In [117]:
df_paises_con_viajes_y_turismo

,id_pais_destino,nombre_pais_destino,nombre_continente,num_viajes_tui_web,num_viajes_segun_viaje_itinerario,total_turistas,total_pernoctaciones
0,1,Albania,Europa,None,None,None,None
1,2,Alemania,Europa,None,None,None,None
2,3,Andorra,Europa,None,None,None,None
3,4,Angola,África,None,None,None,None
4,5,Antigua,América,None,None,None,None
...,...,...,...,...,...,...,...
183,184,Vietnam,Asia,None,None,None,None
184,185,Yemen,Asia,None,None,None,None
185,186,Zambia,África,None,None,None,None
186,187,Zanzibar,África,None,None,None,None


In [118]:
ex.incorporar_información_df_original(df_paises_con_viajes_y_turismo,
                                                                      df_viajes_localizacion_pag_web_agrupado_pais,'id_pais_destino','id_viaje','num_viajes_tui_web')

In [119]:
df_paises_con_viajes_y_turismo

,id_pais_destino,nombre_pais_destino,nombre_continente,num_viajes_tui_web,num_viajes_segun_viaje_itinerario,total_turistas,total_pernoctaciones
0,1,Albania,Europa,5.0,None,None,None
1,2,Alemania,Europa,20.0,None,None,None
2,3,Andorra,Europa,NaN,None,None,None
3,4,Angola,África,NaN,None,None,None
4,5,Antigua,América,2.0,None,None,None
...,...,...,...,...,...,...,...
183,184,Vietnam,Asia,43.0,None,None,None
184,185,Yemen,Asia,NaN,None,None,None
185,186,Zambia,África,NaN,None,None,None
186,187,Zanzibar,África,23.0,None,None,None


In [120]:
df_viajes_localizacion_ciudad_agrupado_pais

,id_pais_destino,id_viaje
0,1,19
1,2,84
2,6,7
3,8,116
4,9,19
...,...,...
111,180,49
112,181,3
113,182,18
114,184,349


In [121]:
ex.incorporar_información_df_original(df_paises_con_viajes_y_turismo,
                                                                      df_viajes_localizacion_ciudad_agrupado_pais,'id_pais_destino','id_viaje','num_viajes_segun_viaje_itinerario')

In [122]:
df_turismo_emisor_agrupado_pais= df_turismo_emisor.groupby('id_pais_destino')[['num_turistas','num_pernoctaciones']].sum().reset_index()

In [123]:
df_turismo_emisor_agrupado_pais

,id_pais_destino,num_turistas,num_pernoctaciones
0,1,180212,1115910
1,2,11949860,68516775
2,3,9323631,24769193
3,4,9669,79896
4,5,220,1900
...,...,...,...
153,183,106829,803330
154,184,130169,1477676
155,185,222,2128
156,186,382,2881


In [124]:
ex.incorporar_información_df_original(df_paises_con_viajes_y_turismo,
                                                                      df_turismo_emisor,'id_pais_destino','num_turistas','total_turistas')

In [125]:
ex.incorporar_información_df_original(df_paises_con_viajes_y_turismo,
                                                                      df_turismo_emisor,'id_pais_destino','num_pernoctaciones','total_pernoctaciones')

In [126]:
df_paises_con_viajes_y_turismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id_pais_destino                    188 non-null    int64  
 1   nombre_pais_destino                188 non-null    object 
 2   nombre_continente                  188 non-null    object 
 3   num_viajes_tui_web                 126 non-null    float64
 4   num_viajes_segun_viaje_itinerario  116 non-null    float64
 5   total_turistas                     158 non-null    float64
 6   total_pernoctaciones               158 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 10.4+ KB


In [127]:
df_paises_con_viajes_y_turismo

,id_pais_destino,nombre_pais_destino,nombre_continente,num_viajes_tui_web,num_viajes_segun_viaje_itinerario,total_turistas,total_pernoctaciones
0,1,Albania,Europa,5.0,19.0,1410.0,5359.0
1,2,Alemania,Europa,20.0,84.0,179330.0,976543.0
2,3,Andorra,Europa,NaN,NaN,120980.0,273540.0
3,4,Angola,África,NaN,NaN,96.0,714.0
4,5,Antigua,América,2.0,NaN,38.0,344.0
...,...,...,...,...,...,...,...
183,184,Vietnam,Asia,43.0,349.0,1486.0,15438.0
184,185,Yemen,Asia,NaN,NaN,111.0,1064.0
185,186,Zambia,África,NaN,NaN,51.0,330.0
186,187,Zanzibar,África,23.0,NaN,NaN,NaN


In [135]:
df_importes_agrupación_competencia_nombrehotel= df_paises_con_viajes_y_turismo.groupby(['nombre_continente'])[['num_viajes_tui_web','num_viajes_segun_viaje_itinerario','total_turistas','total_pernoctaciones']].agg(["mean", "sum", "count"]).reset_index()
df_importes_agrupación_competencia_nombrehotel['Peso_categoria']=round(df_importes_agrupación_competencia_nombrehotel.iloc[:,3]/df_importes_agrupación_competencia_nombrehotel.iloc[:,3].sum()*100,2)
df_importes_agrupación_competencia_nombrehotel['Peso_total_valor_recaudado']=round(df_importes_agrupación_competencia_nombrehotel.iloc[:,2]/df_importes_agrupación_competencia_nombrehotel.iloc[:,2].sum()*100,2)
df_importes_agrupación_competencia_nombrehotel

nombre_continente num_viajes_tui_web               \
                                     mean    sum count   
0              América          16.411765  558.0    34   
1                 Asia          15.925926  430.0    27   
2               Europa           8.850000  354.0    40   
3  No Existe Resultado                NaN    0.0     0   
4              Oceanía           5.857143   41.0     7   
5               África          21.500000  387.0    18   

  num_viajes_segun_viaje_itinerario               total_turistas             \
                               mean     sum count           mean        sum   
0                         84.892857  2377.0    28    5575.964286   156127.0   
1                         97.571429  2732.0    28    1671.976190    70223.0   
2                         49.864865  1845.0    37   61470.071429  2581743.0   
3                         14.000000    14.0     1            NaN        0.0   
4                         43.333333   260.0     6     736.666667     2210.0   
5                        117.937500  1887.0    16    2819.465116   121237.0   

        total_pernoctaciones                  Peso_categoria  \
  count                 mean        sum count                  
0    28         46838.464286  1311477.0    28          26.98   
1    42         11454.190476   481076.0    42          21.43   
2    42        222217.833333  9333149.0    42          31.75   
3     0                  NaN        0.0     0           0.00   
4     3          9484.666667    28454.0     3           5.56   
5    43         10085.511628   433677.0    43          14.29   

  Peso_total_valor_recaudado  
                              
0                      31.53  
1                      24.29  
2                      20.00  
3                       0.00  
4                       2.32  
5                      21.86